# Project Information

Project Background: Building a Desktop and Laptop Support Agent Assistant with Generative AI

## Context

In today's oganizations, providing efficient and effective customer support for technical issues related to desktops and laptops is crucial. Customer support agents often deal with a wide range of queries, from basic troubleshooting steps to more complex hardware and software problems. Access to well-structured and easily searchable support documentation is essential for agents to quickly diagnose and resolve customer issues.


## Problem Outline

Traditional methods of accessing support documentation can be time-consuming. Agents may need to navigate through lengthy PDF files or knowledge base articles to find the relevant information. This can lead to longer resolution times, increased frustration for both agents and customers, and potentially lower customer satisfaction.

##  Proposed Solution


This project aims to leverage the power of Generative AI to create an intelligent assistant that can enhance the capabilities of customer support agents dealing with desktop and laptop issues. The core idea is to process existing support documentation (in this case, a synthetic PDF file created for this purpose) and enable agents to quickly retrieve relevant information and potentially generate helpful responses or troubleshooting steps based on customer queries.

# Key Components:

## Support Documentation

This is the primary dataset. A synthetic PDF file containing common desktop and laptop troubleshooting steps, structured into logical sections (e.g., power issues, display problems, network connectivity, battery issues). This document serves as the knowledge base for the AI assistant.

## Generative AI Model

A Large Language Model (LLM), _command-r-plus_ from _COHERE_ This model will be used to understand user queries and extract relevant information from the support documentation.

## Embedding Model

The embedding model used is _embed-english-light-v3.0_ also from _COHERE_. 

## Vector Store

I have used chromadb in this case. 

## LangChain and Langgraph Framework

The LangChain library was used as the framework to connect the LLM with the support documentation. This involved techniques like:

* Document Loading: Loading and processing the PDF file.
* Text Splitting: Dividing the document into smaller chunks for efficient retrieval.
* Vector Embeddings: Creating vector representations of the text chunks to enable semantic search.
* Retrieval-Based Question Answering: Using a retrieval mechanism (e.g., a vector store and similarity search) to find relevant sections in the documentation based on the agent's query.
* Response Generation: Utilizing the LLM to generate concise and helpful answers or troubleshooting steps based on the retrieved information.

For the agent creation and operation, LangGraph was used.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../input/support-documentation/support_documentation.pdf


In [2]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -qU langchain-core langchain-text-splitters langchain 
!pip install -qU transformers sentence-transformers chromadb pypdf langchain
!pip install -qU "langchain[cohere]"
!pip install -qU langchain-community
!pip install -qU chromadb
!pip install -qU "langchain-chroma>=0.1.2"
!pip install  --upgrade langchain langchain-core langchain-cohere
!pip uninstall -y ypy-websocket
!pip install langchain-core "langgraph>0.2.27" 
!pip install -Uq "unstructured-client<0.30.0"
!pip uninstall -y aiofiles
!pip install "aiofiles>=24.1.0"
!pip install langchain-community
!pip install pypdf
!pip install "unstructured[pdf]"
!pip install chromadb
%config Completer.use_jedi = False

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 4.6 MB/s eta 0:00

In [3]:
from langchain_chroma import Chroma

In [4]:
from kaggle_secrets import UserSecretsClient

CO_API_KEY = UserSecretsClient().get_secret("COHERE_API_KEY")

In [5]:
from langchain.chat_models import init_chat_model

In [6]:
model = init_chat_model("command-r-plus", 
                        model_provider="cohere",
                        cohere_api_key=CO_API_KEY)

In [7]:
!pip install chromadb

# Define loader, text splitters 
The next step is to get the loader and the text spliters so that we can load
and split the text document to get into a array of Documents

In [8]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document


In [9]:
loader = DirectoryLoader(path="..", glob="**/*.pdf", 
                         recursive=True, 
                         show_progress=True)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
docs: Document = loader.load_and_split()

100%|██████████| 1/1 [00:15<00:00, 15.30s/it]


In [10]:
!pip install langchain-chroma


In [11]:
from langchain_cohere.embeddings import CohereEmbeddings
from langchain_cohere.chat_models import ChatCohere

embeddings = CohereEmbeddings(model="embed-english-light-v3.0", cohere_api_key=CO_API_KEY)

In [12]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name = "default_collection",
    embedding_function = embeddings,
    persist_directory="./default_collection_db",
)

retriever = vector_store.as_retriever()

In [13]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(docs))]

In [14]:
doc_ids = vector_store.add_documents(uuids = uuids, documents = docs)
doc_ids

['6ca3e83a-080f-4f8b-b00a-0ec40ac64d27',
 '8ca6e726-5a5e-4081-b135-6da9ba5d2c8b']

In [15]:
# Test the similarity search 
# vector_store.similarity_search("monitor not working", k=2, )

In [16]:
llm = ChatCohere(model="command-r-plus", cohere_api_key=CO_API_KEY)

In [17]:
from langchain_core.prompts import ChatPromptTemplate
retrieval_prompt = ChatPromptTemplate.from_template(
    
    """Answer the user's question based on the context provided below:
    
    Context:
    {context}

    If you are unable to answer based on the document then say "I do not have that
    information with me."
    
    Question: {question}
    
    Answer:"""
)

In [18]:
!pip install langgraph

In [19]:
# Let's do the agent now
from typing import TypedDict, Dict
class AgentState(TypedDict):
    keys: Dict[str, any]


from langgraph.graph import StateGraph, END

In [20]:
state:AgentState = {"keys":{}}
user_input = "My mouse is not working, what might be the problem?"
state = {"keys": {"question": user_input}}
retriever = vector_store.as_retriever(embedding=embeddings, k = 2, )
question = state["keys"]["question"]
retrieved_docs = retriever.get_relevant_documents(question)
len(retrieved_docs)

/tmp/ipykernel_19/2006024849.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(question)


2

In [21]:
workflow = StateGraph(AgentState)

def retrieve_documents(state) -> AgentState:
    """Retrieves relevant documents based on the user's question."""
    question = state["keys"]["question"]
    if question is None:
        raise KeyError(f"No question in state {question}")
    print(f"state is {state}")
    retriever = vector_store.as_retriever(embedding=embeddings, k = 2, )
    retrieved_docs = retriever.get_relevant_documents(question)
    context =  {"context": "\n\n".join([doc.page_content for doc in retrieved_docs])}
    return {"keys":{"question":question, "context":context,}}

def generate_response(state) -> AgentState:
    """Generates the final response using the Cohere model and retrieved context."""
    context = state["keys"]["context"]
    question = state["keys"]["question"]
    # Use the retrieval_prompt here
    response = llm.invoke(retrieval_prompt.format_messages(context=context, question=question))
    state["keys"]["response"] = response
    return state


workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("generate", generate_response)

workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

workflow.set_entry_point("retrieve")

agent = workflow.compile()


In [22]:
state:AgentState = {"keys":{
    "question":"",
    "context":"",
}}
user_input = "My mouse is not working, what might be the problem?"
state = {"keys": {"question": user_input}}
state = agent.invoke(state)

print(state["keys"]["response"])

state is {'keys': {'question': 'My mouse is not working, what might be the problem?'}}
content='The document does not specifically address a non-functional mouse issue. However, based on the general troubleshooting tips provided, here are some potential solutions: \n\n- Restart the Computer: Sometimes, a simple restart can fix temporary software glitches that may be causing the mouse issue.\n\n- Check Connections: Ensure that the mouse is properly connected to the computer. If it\'s a wired mouse, verify that the cable is securely plugged into the appropriate port. If it\'s a wireless mouse, check that the USB receiver is inserted correctly and that the mouse has working batteries.\n\n- Update Drivers: Try updating the mouse driver. You can usually do this by going to the Device Manager in the Control Panel, finding the mouse under "Mice and other pointing devices," and then selecting "Update Driver."\n\n- Try a Different Mouse: If possible, connect a different mouse to your computer t